In [ ]:
import pickle
import nu_smrutils as u
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
class DataPreparation:
    def __init__(self, directory, conditions, epochs):
        self.conditions=conditions
        self.epochs=epochs
        self.directory=directory
    
    def loadConcat(self, subject):
        train_f=self.directory.format(subject, 'train')
        test_f=self.directory.format(subject, 'test')

        train = u.loaddat(train_f)
        tr_df=train.to_data_frame()

        test = u.loaddat(test_f)
        ts_df=test.to_data_frame()
        ts_df['epoch']+=200
        return pd.concat([tr_df, ts_df])

    def convertToSPD(self, df, normalize=True):
        SPD = []
        labels = [] 
        for i in range(self.epochs):
            df_slice=df.loc[df['epoch']==i, :]
            matrix=df_slice.iloc[:, 3:]
            
            if normalize:
                matrix=(matrix-matrix.mean())/matrix.std()
            
            label=df_slice['condition'].iloc[0]

            for j in range(len(self.conditions)):
                if label==self.conditions[j]:
                    label=j #encoding of conditions to integers
                    break
            covmat=matrix.cov().to_numpy()
            SPD.append(covmat)
            labels.append(label) 
        return SPD, labels

    def generateSPDDataset(self, r=[1,54], normalize=True):
        SPDDataset=[]
        labels = []
        for i in range(r[0],r[1]+1):
            df=self.loadConcat(i)
            SPD, l = self.convertToSPD(df,normalize)
            SPDDataset+=SPD
            labels+=l    
        return SPDDataset, labels

In [ ]:
directory = 'datasets/54subjects/Subject{}_{}.pickle'
conditions=['left','right']
subjectRange=[1,54]
epochs = 400